# Intrinsic Periodicity of the SIR System
Continuing our investigation of the Susceptible-Infected-Recovered system, 

$$
\dot{S} = -\frac{\beta*S*I}{N} + \mu N - \mu S\\

\dot{I} = \frac{\beta*S*I}{N} - \gamma I - \mu I\\

\dot{R} = \gamma I - \mu R
$$

With non-trivial endemic equilibrium 

$$
(S^*, \: I^*, \: R^*) = (\frac{1}{R_0}, \:\: \frac{\mu (R_0-1)}{\beta}, \: \: 1-\frac{1}{R_0} - \frac{\mu (R_0-1)}{\beta}) \\

\text{where} \:\: R_0 = \frac{\beta}{\gamma + \mu}
$$

General analysis of a system's stability of and approach to equilibria is beyond the scope of this notebook, and detailed discussions are available elsewhere (e.g., Keeling/Rohani Box 2.4).  In brief, one constructs the Jacobian of the system at the equilibrium points and computes its eigenvalues.  If all eigenvalues have negative real component, then the equilibrium is stable; if the dominant eigenvalues are complex conjugates, then system approaches equilibrium via damped oscillations, with damping constant equal to the real component and frequency equal to the imaginary component. This is the case for the SIR system around the non-trivial (R<sub>0</sub> > 1) equilibrium, with dominant eigenvalues:

$$
\Lambda = -\frac{\mu R_0}{2} \pm \frac{\sqrt{\mu^2 R_0^2 - \frac{4}{A G}}}{2} \\
\text{where} \:\: A = \frac{1}{\mu (R_0 -1)} and G = \frac{1}{\mu + \gamma}

$$

In general, $\mu^2 R_0^2$ μ^2R<sub>0</sub><sup>2</sub> is quite small, and the intrinsic periodicity of the system is 
$$T \approx 2 \pi \sqrt{A G}$$.

### Construct the model
The model is constructed as in notebook 05.  As again, we are looking at behavior around the endemic equilibrium, the same considerations of large-ish populations and long simulations apply here. 

### Sanity check
The first test, as always, ensures that certain basic constraints are being obeyed by the model.  As it stands, I am not actually explicitly tracking the recovered population, but I can check that 
$$S_t = N_t - \sum{\Delta_I}$$.  

### Scientific test
The scientific test will sample a set of (μ, γ, R<sub>0</sub>) tuplets and confirm that the periodicity is   

### Future work
The addition of an exposed compartment with rate constant σ should change this result, by changing the generation time *G* to 
$$\frac{1}{\mu + \gamma} + \frac{1}{\mu + \sigma}$$


In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from laser_core.propertyset import PropertySet
from scipy.ndimage import gaussian_filter1d
from scipy.signal import find_peaks

from laser_generic import Births_ConstantPop
from laser_generic import Infection
from laser_generic import Model
from laser_generic import Susceptibility
from laser_generic import Transmission
from laser_generic.importation import Infect_Random_Agents
from laser_generic.utils import seed_infections_randomly
from laser_generic.utils import set_initial_susceptibility_randomly

f"{np.__version__=}"

OK, in testing here, we run into a couple of challenges, and my training in spectral
analysis is a bit old at this point.  Mainly, we're interested in relatively low-frequency
signals relative to our vector length - periods that can be order a few years in a sequence of a few decades,
so only a few oscillations.  Not impossible but tough
Second is that there are lots of other signals that will probably settle down over
time, but may not.  Lastly, there seem to be some windowing effects from the length of the
vector itself.
So some approaches to get arounds this:
Try to start relatively close to equilibrium
Use time-domain autocorrelation spectrum 
Instead of looking for the period of maximum power, specifically look for a peak in the vicinity of 
the expected period.


In [ ]:
scenario = pd.DataFrame(data=[["homenode", 2e6]], columns=["name", "population"])
parameters = PropertySet(
    {"seed": 4, "nticks": 36500, "verbose": True, "beta": 0.4, "inf_mean": 12, "cbr": 45, "importation_period": 180, "importation_count": 3}
)

In [ ]:
model = Model(scenario, parameters)
model.components = [
    Infect_Random_Agents,
    Births_ConstantPop,
    Susceptibility,
    Infection,
    Transmission,
]

seed_infections_randomly(model, ninfections=1)
model.run()
plt.plot(model.patches.cases)

In [ ]:
model.patches.cases[10000:10010]

### Sanity checks
As always, check that we haven't broken anything - S+I+R = N at all times

In [ ]:
cases = np.squeeze(model.patches.cases)
susc = np.squeeze(model.patches.susceptibility)
rec = np.squeeze(model.patches.recovered)
inc = np.squeeze(model.patches.incidence)
births = np.squeeze(model.patches.births)
pops = np.squeeze(model.patches.populations)[:-1]

print("S+I+R = N:  " + str(np.isclose(cases + susc + rec, pops).all()))

In [ ]:
mu = (1 + model.params.cbr / 1000) ** (1 / 365) - 1
R0 = model.params.beta / (1 / model.params.inf_mean + mu)
A = 1 / ((R0 - 1) * mu) / 365
G = 1 / (mu + 1 / model.params.inf_mean) / 365
T_exp = 2 * np.pi * np.sqrt(A * G)


def ID_freq_peakfinder(y0, T_exp, cutoff=18250, plot=False):
    y = y0[cutoff:]
    y = y - np.mean(y)
    y = gaussian_filter1d(y, sigma=100)
    peaks, _ = find_peaks(y, distance=T_exp * 365 / 2)
    if plot:
        plt.figure()
        plt.plot(y, alpha=0.5)
        plt.plot(peaks, y[peaks], "x")
    return np.median(np.diff(peaks)) / 365


T_obs_pf = ID_freq_peakfinder(np.squeeze(model.patches.cases), T_exp, plot=True)


def ID_freq_autocorr(y0, cutoff=18250):
    # Compute the FFT
    Y1 = np.fft.fft(y0[cutoff:] - np.mean(y0[cutoff:]))

    # Compute the circular autocorrelation using the inverse FFT
    circular_autocorr = np.fft.ifft(Y1 * np.conj(Y1)).real
    # Plot only the positive frequency spectrum
    peaks, _ = find_peaks(circular_autocorr, distance=300)
    return peaks[0] / 365


T_obs_fft = ID_freq_autocorr(np.squeeze(model.patches.cases))

plt.text(0.05, 0.9, f"T expected: {T_exp:.2f} y", transform=plt.gca().transAxes)
plt.text(0.05, 0.85, f"T observed, peakfinding: {T_obs_pf:.2f} y", transform=plt.gca().transAxes)
plt.text(0.05, 0.8, f"T observed, FFT: {T_obs_fft:.2f} y", transform=plt.gca().transAxes)
plt.show()

### Larger test suite
OK, so now we are going to replicate the above test for many values of R0 and cbr, as a scientific validity test.

TODO: As written, this takes a long time.  That's because the space of random values being sampled occasionally produces models with long equilibration times and long periodicities.  So as of right now, these run for 100 years to sidestep this.  Should instead restrict the range of values - higher cbrs and R<sub>0</sub>s - so that the average age at infection is consistently lower and we can get away with running for like 30-50 years instead.  

In [ ]:
# import os

nsims = 10
nticks = 36500
cbrs = 30 + 50 * np.random.rand(nsims)
inf_means = 5 + 45 * np.random.rand(nsims)
R0s = 2.5 + 7.5 * np.random.rand(nsims)
mu = [((1 + cbr / 1000) ** (1 / 365) - 1) for cbr in cbrs]
A = [1 / ((R0 - 1) * mu) / 365 for R0, mu in zip(R0s, mu)]
G = [1 / (mu + 1 / inf_mean) / 365 for mu, inf_mean in zip(mu, inf_means)]
T_exp = [2 * math.pi * np.sqrt(A * G) for A, G in zip(A, G)]
mycases = np.zeros((nsims, nticks))
params_df = pd.DataFrame(
    {
        "cbr": cbrs,
        "inf_mean": inf_means,
        "R0": R0s,
        "A": A,
        "G": G,
        "T_exp": T_exp,
    }
)
i = 0
for cbr, inf_mean, R0 in zip(cbrs, inf_means, R0s):
    mu = (1 + cbr / 1000) ** (1 / 365) - 1
    parameters = PropertySet(
        {
            "seed": 2,
            "nticks": 36500,
            "verbose": True,
            "beta": R0 * (mu + 1 / inf_mean),
            "inf_mean": inf_mean,
            "cbr": cbr,
            "importation_period": 180,
            "importation_count": 3,
        }
    )
    model = Model(scenario, parameters)
    model.components = [
        Births_ConstantPop,
        Susceptibility,
        Transmission,
        Infection,
        Infect_Random_Agents,
    ]

    set_initial_susceptibility_randomly(model, 1 / R0 + 0.02)
    seed_infections_randomly(model, ninfections=1)
    model.run()
    plt.plot(model.patches.cases)
    mycases[i] = np.squeeze(model.patches.cases)
    i = i + 1

# output_folder = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "laser-generic-outputs", "periodicity"))
# os.makedirs(output_folder, exist_ok=True)
# params_df.to_csv(os.path.join(output_folder, "params_df.csv"), index=False)
# np.save(os.path.join(output_folder, "mycases.npy"), mycases)

In [ ]:
params_df["T_obs_peakfinder"] = np.nan
params_df["T_obs_autocorr"] = np.nan

for i in range(mycases.shape[0]):
    params_df.loc[i, "T_obs_peakfinder"] = ID_freq_peakfinder(np.squeeze(mycases[i, :]), params_df.loc[i, "T_exp"], plot=False)
    params_df.loc[i, "T_obs_autocorr"] = ID_freq_autocorr(np.squeeze(mycases[i, :]))

In [ ]:
params_df

In [ ]:
plt.plot(params_df["T_exp"], params_df["T_obs_peakfinder"], "o")